In [6]:
import csv
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
book_comments = {}  # 用于存储书名及其对应的评论内容
def load_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        # 创建一个 DictReader 对象，用于按字典形式读取 CSV 文件，指定分隔符为制表符
        reader = csv.DictReader(file, delimiter='\t')
        # 遍历 CSV 文件中的每一行
        for row in reader:
            # print(row['book'], row['body'])
            # break
            book = row['book']  # 从当前行中提取书名
            comment = row['body']  # 从当前行中提取评论内容
            comment_words = jieba.lcut(comment)   # 使用 jieba 库对评论内容进行分词处理
            # print(book, comment_words)
            # break
            if book == '':
                continue
            book_comments[book] =book_comments.get(book, [])
            book_comments[book].append(comment_words)
    return book_comments

if __name__ == '__main__':

    stop_words = [line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')]  # 读取停用词表
    missing_tokens = ['ain', 'aren', 'couldn', 'daren', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn']
    stop_words.extend(missing_tokens)

    book_comments = load_data("doubanfix_comments.txt") # 加载豆瓣评论数据
    # print(len(book_comments))


    book_names = []
    book_data = []
    for book_name, comments in book_comments.items():
       book_names.append(book_name)
       book_data.append(comments)

    corpus = [' '.join([' '.join(words) for words in bookdata]) for bookdata in book_data]
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(corpus)
    # print(tfidf_matrix.shape)

    similarity_matrix = cosine_similarity(tfidf_matrix)

    book_list = list(book_comments.keys())
    # print(book_list)
    book_name = input("请输入书名：")
    try:
      book_index = book_names.index(book_name)
      recommend_book_index = np.argsort(-similarity_matrix[book_index])[1:11]
      for idx in recommend_book_index:
        print(f"《 {book_names[idx]} 》 \t 相似度：{similarity_matrix[book_index][idx]:.4f}")
    except ValueError:
      print("输入的书名不在数据集！")
    print()

          

《 鬼吹灯之精绝古城 》 	 相似度：0.6428
《 鬼吹灯之云南虫谷 》 	 相似度：0.5621
《 盗墓笔记3 》 	 相似度：0.5109
《 盗墓笔记4 》 	 相似度：0.4439
《 盗墓笔记2 》 	 相似度：0.3613
《 达·芬奇密码 》 	 相似度：0.1587
《 哈利·波特与魔法石 》 	 相似度：0.1349
《 何以笙箫默 》 	 相似度：0.1311
《 大地之灯 》 	 相似度：0.1283
《 时间旅行者的妻子 》 	 相似度：0.1281

